In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('scrape-books-from-goodreads')
jovian.set_colab_id('1x9NOCCNlBSmL82ih-K0SNt_xJMkY-NOq')

     |████████████████████████████████| 68 kB 5.2 MB/s 


In [2]:
!pip install jovian --upgrade --quiet

In [3]:
import jovian

In [4]:
# Execute this to save new versions of the notebook
jovian.commit(project="scrape-books-from-goodreads")

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/prasanthi-vvit/scrape-books-from-goodreads


'https://jovian.ai/prasanthi-vvit/scrape-books-from-goodreads'

In [5]:
# RUN THIS CELL WHEN USING THE NOTEBOOK ON COLAB - NO PREVIOUS INSTALLATION OF SELENIUM IS NEEDED
# # install chromium, its driver, and selenium
!apt update -q
!apt install chromium-chromedriver -q
!pip install selenium -q
# set options to be headless
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,831 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
H

In [7]:
from constants import user_id,password
import pandas as pd

In [10]:
def get_goodreadsbooks(driver):
  driver.get("https://www.goodreads.com/user/sign_in")
  user_name = driver.find_element(By.ID,'user_email')
  user_name.send_keys(user_id)
  password_box = driver.find_element(By.ID,'user_password')
  password_box.send_keys(password)
  driver.find_element(By.CLASS_NAME,'submitPara')
  submit_button = driver.find_element(By.XPATH,"//input[@name='next'][@type='submit']")
  submit_button.click()
  time.sleep(2)
  books_tag=driver.find_element(By.LINK_TEXT,'My Books')
  books_tag.click()
  want_a_tag=driver.find_element(By.XPATH,'//*[@id="paginatedShelfList"]/div[3]/a')
  driver.get(want_a_tag.get_attribute('href'))
  books_data=[]
  for table in driver.find_elements(By.XPATH,'//*[contains(@id,"books")]//tr'):
    data1 = [item.text for item in table.find_elements(By.XPATH,".//*[self::td or self::th]")]
    books_data.append(data1)
  driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/div/header/div[2]/div/div[3]/ul/li[5]').click()
  driver.find_element(By.LINK_TEXT,'Sign out').click()
  return books_data

In [11]:
def get_goodreadscsv(books_data):
  dict1={'Title':[],'Author':[],'Avg_rating':[]}
  for i in range(1,len(books_data)):
    dict1['Title'].append(books_data[i][3]);
    dict1['Author'].append(books_data[i][4]);
    dict1['Avg_rating'].append(books_data[i][9]);
  goodreads_df=pd.DataFrame(dict1);
  goodreads_df.to_csv('books_goodreads.csv',header=list(dict1.keys()),index=None);
  print('books_goodreads.csv is generated.')
  return goodreads_df

In [28]:
def get_amazonprice(goodreads_df):
  dict2={'Title':[],'Ratings':[],'Price':[]}
  for title in goodreads_df['Title']:
    driver.get("https://www.amazon.in/")
    input=driver.find_element(By.ID,"twotabsearchtextbox")
    input.send_keys(title)
    input.send_keys(Keys.ENTER)
    time.sleep(2)
    new_page=driver.find_element(By.PARTIAL_LINK_TEXT,title)
    new_page.click()
    driver.get(new_page.get_attribute('href'))
    dict2['Title'].append(driver.find_element(By.XPATH,'//*[@id="productTitle"]').text)
    dict2['Ratings'].append(driver.find_element(By.ID,'acrCustomerReviewLink').text)
    dict2['Price'].append(driver.find_element(By.CLASS_NAME,'a-color-price').text)
  amazon_df=pd.DataFrame(dict2)
  amazon_df.to_csv('books_amazon.csv',header=['Title','Ratings','Price'],index=None);
  print('books_amazon.csv is generated')
  return amazon_df

In [14]:
def get_finalcsv(goodreads_df,amazon_df):
  books_amazon_goodreads_df=goodreads_df.merge(amazon_df, on='Title')
  books_amazon_goodreads_df.to_csv('books_amazon_goodreads.csv')
  print('books_amazon_goodreads.csv is generated')

In [27]:
books_data=get_goodreadsbooks(driver)

In [29]:
goodreads_df=get_goodreadscsv(books_data)

books_goodreads.csv is generated.


In [30]:
amazon_df=get_amazonprice(goodreads_df)

books_amazon.csv is generated


In [31]:
get_finalcsv(goodreads_df,amazon_df)


books_amazon_goodreads.csv is generated


In [41]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/prasanthi-vvit/scrape-books-from-goodreads


'https://jovian.ai/prasanthi-vvit/scrape-books-from-goodreads'